In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from scipy import signal
import sys

In [2]:
def GetRMSE(x2, y2, x1, y1):
    from scipy.spatial.distance import pdist

    er = []
    for idx in range(len(x2)):
        X = np.asarray([[x1[idx], y1[idx]], 
                         [x2[idx], y2[idx]]])
        temp_er = pdist(X,metric = 'euclidean')
        er.append(temp_er[0])
    er = np.asarray(er)
    return(er)

In [3]:
# parameters used to filter data

cutoff = 24

interpol_order = 3 # order for polynomial interpolation

# win_gauss = signal.gaussian(10,3) # gaussian kernal for smoothening interpolated data

# parameters for the savitzky-golay filter
savgol_win = 11
savgol_polyorder = 3

# window_length=7, polyorder=2,

# ROLLING_WINDOW = [11, 21, 31][0]

### remove x y points based on threshold ( of rmse distance from previous frame ) 

In [4]:
direc = r"../dataFolders/PaperPipelineOutput/RawTracks/"
visitnum = ['FirstVisit/', 'Later7thVisit/','LaterVisit/', 'LastVisit/']

for visit in visitnum[1:2]:
    path = os.path.join(direc, visit)
    trackslist = glob.glob(path + '*.csv')

In [5]:
circ_parameters_path = glob.glob('../dataFolders/PaperPipelineOutput/CircleParameters/' + '*.csv')
circ_parameters = pd.read_csv(circ_parameters_path[0])

In [6]:
outpath = os.path.join('../dataFolders/PaperPipelineOutput/FilteredTracks_v2/', visit)
print(outpath)

outpathfig = os.path.join('../dataFolders/PaperPipelineOutput/Figures/v2/FilteredTracks/', visit)
print(outpathfig)

../dataFolders/PaperPipelineOutput/FilteredTracks_v2/Later7thVisit/
../dataFolders/PaperPipelineOutput/Figures/v2/FilteredTracks/Later7thVisit/


In [ ]:
for data in trackslist:
#     data = trackslist[0]
    name = os.path.basename(data)[:-4]
    print('working on ' + name)
    
    # remove problem cases
    # for LaterVisit/
#     if name in ['c-1_m1_visit_19', 'c-2_m2_visit_19']:
    # for LastVisit/
#     if name in ['c-10_m1_visit_-1', 'c-1_m2_visit_-1', 'c-2_m2_visit_-1', 'c-3_m21_cropped_visit_-1']:
#         continue
    # for Later7thVisit/
    if name in ['c-10_m2_visit_6', 'c-1_m1_visit_6']:
        continue
        
    file = pd.read_csv(data)
    x = file.x.values
    y = file.y.values
    p = file.likelihood

    x_notinView = x <=5
    y_notinView = y <=5

    x[x_notinView & y_notinView]=np.nan
    y[x_notinView & y_notinView]=np.nan


    # add filter for DLC likelihood
    med = file['likelihood'].rolling(11).median()
    x[med < 0.6] = np.nan
    y[med < 0.6] = np.nan

    if x.size == 0 or y.size == 0:
        print(name + 'has emtpy x y tracks')
        continue
    
    name = [n for n in circ_parameters.name if n + '_' in data][0]

    circ_x = circ_parameters.loc[circ_parameters.name == name, 'circ_x'].values
    circ_y = circ_parameters.loc[circ_parameters.name == name, 'circ_y'].values
    circ_radii = circ_parameters.loc[circ_parameters.name == name, 'circ_radii'].values
    
    # get rmse values for subsequent frames
    rmse = GetRMSE(x[1:], y[1:], x[:-1], y[:-1])

    filtered_x = np.copy(x[1:])
    filtered_y = np.copy(y[1:])

    filtered_x[(rmse > cutoff) | (rmse == np.nan)] = np.nan
    filtered_y[(rmse > cutoff) | (rmse == np.nan)] = np.nan

    filtered_r = np.linalg.norm([filtered_x - circ_x, filtered_y - circ_y], axis = 0)
    filtered_r = filtered_r/circ_radii
    filt_trajectory = pd.DataFrame([filtered_x, filtered_y, filtered_r]).T
    filt_trajectory.columns = ['x', 'y', 'r']
    
    t = (pd.Series(filtered_x).rolling(30).median(center=True))
    t_reverse = t[::-1]
    t_reverse
    s = np.argmax([ ~np.isnan(t) for t in t_reverse ] )
    trim = len(t)-s
    
    # Apply filters

    trajectory = filt_trajectory.copy()
    trajectory = trajectory.loc[0:trim, :]
    print(trajectory.shape)

    for colname in trajectory.columns:
#         print(colname)
        trajectory.loc[:, colname] = signal.medfilt(trajectory.loc[:, colname], kernel_size=11)
        trajectory.loc[:, colname] = trajectory.loc[:, colname].interpolate(method = 'polynomial', order = 3, limit = 40)
        
        nans = trajectory.loc[:,colname].isnull()
        trajectory.loc[:,colname] = trajectory.loc[:,colname].interpolate(method = 'pad')
        trajectory.loc[:, colname] = signal.savgol_filter(trajectory.loc[:, colname],
                                                          window_length=savgol_win,
                                                          polyorder=savgol_polyorder,
                                                          axis=0)
        trajectory.loc[nans, colname]= np.nan
        
    trajectory_r = np.linalg.norm([trajectory.loc[:,'x'].values - circ_x, trajectory.loc[:,'y'].values - circ_y], axis = 0)
    trajectory['r'] = trajectory_r/circ_radii
    
#     fig = plt.figure()
    axes = pd.concat([filt_trajectory, trajectory], axis = 1).plot(subplots = True, figsize = (15,8))
    fig = plt.gcf()
#     .get_figure()
    fig.savefig(outpathfig + name + '_' + visit[:-1] + '.pdf')
    plt.close() 
        
    trajectory.to_csv(outpath + name + '_' + visit[:-1] + '.csv')

working on c-10_m11_visit_6
(299, 3)


C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in greater
C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in greater


working on c-10_m12_visit_6
(1087, 3)


C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in greater
C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in greater


working on c-10_m13_visit_6
(278, 3)


...

working on c-10_m15_visit_6
(972, 3)


C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in greater
C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in greater


working on c-10_m16_cropped_visit_6
(1960, 3)


C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:49: RuntimeWarning: invalid value encountered in greater
C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in greater


working on c-10_m17_cropped_visit_6
